# Provisioning a bare metal node on Chameleon

- **Estimated time**: 30 minutes
- **Requirements**:
  - Active Chameleon allocation
  - At least one node and public IP address free on the testbed

Let's walk through the steps of provisioning a bare metal node on Chameleon. This tutorial will show you how to make a reservation for a particular node (called a Lease), and then provision the node with a Chameleon-provided [CentOS 7 disk image](https://www.chameleoncloud.org/appliances/1/) that allows you to log in via SSH. We will also show how to attach a public IP to your running instance so that SSH is available over the public internet (by default, there is no connectivity to an experimental environment from the public Internet.)

1. [Step 1: Authenticate with your Chameleon project](#Step-1%3A-Authenticate-with-your-Chameleon-project)
1. [Step 2: Create a reservation](#Step-2%3A-Create-a-reservation)
1. [Step 3: Provision a new bare metal instance](#Step-3%3A-Provision-a-new-bare-metal-instance)
1. [Step 4: Assign a public IP address](#Step-4%3A-Assign-a-public-IP-address)

### Variables you'll see/use in this Notebook

  - `NODE_TYPE`: the type of bare metal node on Chameleon to reserve (see [list of all types](https://chameleoncloud.readthedocs.io/en/latest/technical/reservations.html#chameleon-node-types))
  - `LEASE_NAME`: the name of your lease
  - `KEYPAIR_NAME`: the name of an SSH keypair used to authenticate to your instance
  - `SERVER_NAME`: the name of your instance
  - `FLOATING_IP`: the public IP address of your instance

In [ ]:
NODE_TYPE=compute_haswell
LEASE_NAME="$USER-tutorial-$(date +%b%d)"
KEYPAIR_NAME="$USER-jupyter-$(hostname)"
SERVER_NAME="$LEASE_NAME"
# FLOATING_IP will be set later; it is dynamically reserved

## Step 1: Authenticate with your Chameleon project

This requires that you have a project with an active allocation on Chameleon. If you do not have an active allocation, you should either [create a new project](https://chameleoncloud.readthedocs.io/en/latest/technical/project.html#creating-a-project) or [extend an existing allocation](https://chameleoncloud.readthedocs.io/en/latest/technical/project.html#recharge-or-extend-your-allocation) (Note: this requires having PI status! Ask your PI to perform these tasks if you are not a PI.)

In [ ]:
if [[ "${OS_PROJECT_NAME:+x}" != "x" ]]; then
  echo "No project could automatically be detected. Overriding in Notebook."
  # For example...
  export OS_PROJECT_NAME='CH-000000'
fi

# Test that you can authenticate
openstack token issue >/dev/null && echo "Successfully authenticated to project $OS_PROJECT_NAME"

## Step 2: Create a reservation

We're actually going to reserve two things on Chameleon: a bare metal node, and a public IP address. While Chameleon has several nodes and IP addresses for use by researchers, the supply is still constrained relative to the amount of people using the system, so, unlike commercial clouds like AWS, it is important to reserve them ahead of time. In practice there are usually enough commodity hardware that you can use for exploration right away.

We will be creating the lease using the `blazar` CLI utility. Blazar is the name of the system that manages leases for Chameleon. The CLI `lease-create` command takes a few arguments:

  - `--physical-reservation`: used whenever you are reserving a bare metal node. This takes a list of properties to apply to the reservation: a `min` and `max` number of nodes (these can be the same), and a `resource_properties` selector that defines how to pick the nodes from the inventory.
  - `--reservation`: used for reserving things other than bare metal nodes. This also takes a list of properties: `resource_type` denotes the type of thing being reserved, e.g. "virtual:floatingip" for a public IP address, and then a list of required properties for that type, e.g. `network_id` and `amount` are required for the "virtual:floatingip" type, the former denoting which Network to allocate the IP from, the latter indicating how many IPs.
  - `--start-date`: finally an easy one! Denotes when the lease should start (YYYY-mm-dd HH:MM format)
  - `--end-date`: when the lease should end. If not defined, defaults to one day from the start date.
  
It is possible to specify only `--physical-reservation` if only a bare metal node is desired, and similarly, only `--reservation` if only some other type of reservation is desired. The following example utilizes both, because we want to reserve an IP address and a node.

In [ ]:
PUBLIC_NETWORK_ID=$(openstack network show public -f value -c id)

blazar lease-create \
  --physical-reservation min=1,max=1,resource_properties="[\"=\", \"\$node_type\", \"$NODE_TYPE\"]" \
  --reservation "resource_type=virtual:floatingip,network_id=$PUBLIC_NETWORK_ID,amount=1" \
  --start-date "$(date +'%Y-%m-%d %H:%M')" \
  --end-date "$(date +'%Y-%m-%d %H:%M' -d'+1 day')" \
  "$LEASE_NAME"

The lease should start quickly, but it can take a minute or two, depending on how busy the system is. You can use the `wait_lease` helper to pause the Notebook until the lease is ready.

In [ ]:
wait_lease "$LEASE_NAME"

## Step 3: Provision a new bare metal instance

Now let's use that reserved node and launch a bare metal instance on it. You can launch existing images maintained by Chameleon. It's also possible to create snapshots of your current working environment on Chameleon as a new image, and launch that image in the future, allowing you to avoid having to bootstrap all of your software and special setup in the future! We'll see how to do that later.

### Create an SSH keypair

All Chameleon images use SSH keypair authentication, as opposed to password authentication, for additional security. Remember that SSH is going to be exposed to the public Internet! Don't say I didn't warn you...

Your Jupyter Notebook already has a SSH key generated at `$HOME/.ssh` - we can use that one.

In [ ]:
openstack keypair create --public-key ~/.ssh/id_rsa.pub "$KEYPAIR_NAME"

### Launch an instance

We're now ready to launch an instance. In order to do so, we need to choose a Network to connect it to (by default, we will use `sharednet1`, a default network that is shared by default with all projects; if you need proper isolation please refer to the [Isolated Network VLANs documentation](https://chameleoncloud.readthedocs.io/en/latest/technical/networks/networks_vlan.html).)

As with the Keypair creation, we are using the `openstack` CLI here. This CLI can be used to inspect and modify most entities on the Chameleon testbed. Most of the [official documentation for OpenStack](https://docs.openstack.org/python-openstackclient/rocky/cli/command-list.html), upon which Chameleon is based, applies if you want to learn more about what is possible.

To make things a bit easier for you, we've included a helper function `lease_server_create_default_args`, which returns a list of arguments you can feed in to the `openstack server create` command. In particular, it adds a special `--hint` flag that is used to indicate which lease you are launching the instance under.

In [ ]:
SERVER_NAME="$LEASE_NAME"

openstack server create --key-name "$KEYPAIR_NAME" \
  $(lease_server_create_default_args "$LEASE_NAME") \
  $SERVER_NAME

### Wait for instance to spawn... this can take a while.

This is expected to take about 10 minutes. A lot is happening behind the scenes to get your image running on that machine! In case you want to wait in the Notebook, you can use the `wait_instance` helper function--just pass it the instance name (or instance UUID).

In [ ]:
wait_instance "$SERVER_NAME"

## Step 4: Assign a public IP address

By default, your new instance is assigned only an internal private IP, and is not reachable from the public internet. In order to reach your node, you must attach a public IP (called a "Floating IP", because it can "float" between instances over time; they are shared resources.)

We will attach the public IP that was reserved for us. This information is a bit tricky to get just out of the `openstack` or `blazar` CLI, so we've provided a `lease_list_floating_ips` helper function for you. It will print a list of public IPs reserved as part of your lease.


In [ ]:
FLOATING_IP=$(lease_list_floating_ips "$LEASE_NAME" | head -n1)

openstack server add floating ip "$SERVER_NAME" "$FLOATING_IP" \
  && echo "Attached floating ip $FLOATING_IP!"

Once the public IP is attached, it may still take a short bit of time (~1-2 minutes) for the routing rules to propagate throughout the system. You can use the `wait_ssh` helper to wait until SSH is available on your public IP.

In [ ]:
wait_ssh "$FLOATING_IP"

Finally, let's SSH in to the node, using the SSH keypair installed on the Jupyter Notebook server. You must log in as the `cc` user. You can also open a new Terminal in the JupyterLab interface (`File`... `New`... `Terminal`) and open an interactive SSH session that way, if you like.

In [ ]:
ssh cc@"$FLOATING_IP" echo 'User $(whoami) connected on $(hostname)!'

## Recap

In this tutorial, you learned how to create a lease for resources on the Chameleon testbed using the `blazar` CLI in Bash. You learned how to create a combined lease for both a floating IP address and also a bare metal node. You also got a (brief) introduction to the `openstack` CLI, which can be used to inspect and control most of the testbed. Chameleon runs on top of OpenStack, so most public documentation on OpenStack also applies to Chameleon. Finally, you learned how to assign a public IP to your launched instance, and (hopefully!) were able to log in to the node via SSH and your private SSH keypair stored in your Jupyter Notebook server.